# 黑板 - Blackboards

## 说明

`Blackboards`不是行为树必要组件，但`Blackboards`是行为树之前共享数据常见机制。

根据框架需要，实现有很大的差异。最简单的实现采用全局访问，以键值形式存储。更严格的实现有范围访问、连接数据端口。

- 类
```python
class py_trees.blackboard.Client(*, name=None, namespace=None)
```

## 示例

- 创建客户端

In [6]:
import py_trees
provided = py_trees.blackboard.Client(name="Provided")
# print(provided)
generated = py_trees.blackboard.Client()
# print(generated)

- 为黑板上的键注册读/写访问权

In [3]:
blackboard = py_trees.blackboard.Client(name="Client")
blackboard.register_key(key="foo", access=py_trees.common.Access.WRITE)
blackboard.register_key(key="bar", access=py_trees.common.Access.READ)
blackboard.foo = "hello"
print(blackboard)

Blackboard Client
  Client Data
    name              : Client
    namespace         : /
    unique_identifier : 28b755e0-003e-4de0-9579-842ce4669ab6
    read              : {'/bar'}
    write             : {'/foo'}
    exclusive         : set()
  Variables
    /foo : hello
    /bar : -



- 连接规则

In [1]:
import py_trees

def check_foo():
    blackboard = py_trees.blackboard.Client(name="Reader")
    blackboard.register_key(key="foo", access=py_trees.common.Access.READ)
    print("Foo: {}".format(blackboard.foo))


blackboard = py_trees.blackboard.Client(name="Writer")
blackboard.register_key(key="foo", access=py_trees.common.Access.WRITE)
blackboard.foo = "bar"
check_foo()

Foo: bar


- 复杂对象

In [7]:
class Nested(object):
    def __init__(self):
        self.foo = None
        self.bar = None

    def __str__(self):
        return str(self.__dict__)


writer = py_trees.blackboard.Client(name="Writer")
writer.register_key(key="nested", access=py_trees.common.Access.WRITE)
reader = py_trees.blackboard.Client(name="Reader")
reader.register_key(key="nested", access=py_trees.common.Access.READ)

writer.nested = Nested()
writer.nested.foo = "I am foo"
writer.nested.bar = "I am bar"

foo = reader.nested.foo
print(writer)
print(reader)

Blackboard Client
  Client Data
    name              : Writer
    namespace         : /
    unique_identifier : 082c7218-e0bf-4d70-9bd7-1c3a52ac8eb1
    read              : set()
    write             : {'/nested'}
    exclusive         : set()
  Variables
    /nested : {'foo': 'I am foo', 'bar': 'I am bar'}

Blackboard Client
  Client Data
    name              : Reader
    namespace         : /
    unique_identifier : b3ee9a99-0bb2-442d-ab3c-23baa940f43f
    read              : {'/nested'}
    write             : set()
    exclusive         : set()
  Variables
    /nested : {'foo': 'I am foo', 'bar': 'I am bar'}



- 记录显示活动流

In [32]:
# 开启记录流
py_trees.blackboard.Blackboard.enable_activity_stream(maximum_size=100)
reader = py_trees.blackboard.Client(name="Reader")
reader.register_key(key="foo", access=py_trees.common.Access.READ)
writer = py_trees.blackboard.Client(name="Writer")
writer.register_key(key="foo", access=py_trees.common.Access.WRITE)

In [25]:
writer.foo = "bar"

In [29]:
writer.foo = "hello"

In [30]:
unused_result = reader.foo

In [33]:
# 显示流
print(py_trees.display.unicode_blackboard_activity_stream())

Blackboard Activity Stream
    /foo : WRITE         | Writer | → bar
    /foo : WRITE         | Writer | → hello
    /foo : WRITE         | Writer | → hello
    /foo : WRITE         | Writer | → hello
    /foo : READ          | Reader | ← hello


In [34]:
# 清除记录
py_trees.blackboard.Blackboard.activity_stream.clear()

- 只显示其中一部分

In [36]:
writer = py_trees.blackboard.Client(name="Writer")
for key in {"foo", "bar", "dude", "dudette"}:
    writer.register_key(key=key, access=py_trees.common.Access.WRITE)

reader = py_trees.blackboard.Client(name="Reader")
for key in {"foo", "bar"}:
    reader.register_key(key="key", access=py_trees.common.Access.READ)

writer.foo = "foo"
writer.bar = "bar"
writer.dude = "bob"

# all key-value pairs
print(py_trees.display.unicode_blackboard())
# various filtered views
print(py_trees.display.unicode_blackboard(key_filter={"foo"}))
print(py_trees.display.unicode_blackboard(regex_filter="dud*"))
print(py_trees.display.unicode_blackboard(client_filter={reader.unique_identifier}))
# list the clients associated with each key
# print(py_trees.display.unicode_blackboard(display_only_key_metadata=True))

Blackboard Data
    /bar    : bar
    /dude   : bob
    /dudette: -
    /foo    : foo
    /key    : -
    /nested : {'foo': 'I am foo', 'bar': 'I am bar'}

Blackboard Data
  Filter: '{'foo'}'

Blackboard Data
  Filter: 'dud*'
    /dude   : bob
    /dudette: -

Blackboard Data
  Filter: {UUID('f1c4ea86-f9f2-4bc1-ae8d-9e46a1730492')}
    /key: -



- 命名空间

In [70]:
blackboard = py_trees.blackboard.Client(name="Global")
parameters = py_trees.blackboard.Client(name="Parameters", namespace="parameters")

blackboard.register_key(key="foo", access=py_trees.common.Access.WRITE)
blackboard.register_key(key="/bar", access=py_trees.common.Access.WRITE)
blackboard.register_key(key="/parameters/default_speed", access=py_trees.common.Access.WRITE)
parameters.register_key(key="aggressive_speed", access=py_trees.common.Access.WRITE)

blackboard.foo = "foo"
blackboard.bar = "bar"
blackboard.parameters.default_speed = 20.0
parameters.aggressive_speed = 60.0

miss_daisy = blackboard.parameters.default_speed
van_diesel = parameters.aggressive_speed

print(blackboard)
print(parameters)

Blackboard Client
  Client Data
    name              : Global
    namespace         : /
    unique_identifier : 6465755d-b56a-4f8e-b816-298d128ea3e6
    read              : set()
    write             : {'/foo', '/parameters/default_speed', '/bar'}
    exclusive         : set()
  Variables
    /foo                      : foo
    /bar                      : bar
    /parameters/default_speed : 20.0

Blackboard Client
  Client Data
    name              : Parameters
    namespace         : /parameters
    unique_identifier : 62b01d41-c388-4af2-96f3-189142d582e9
    read              : set()
    write             : {'/parameters/aggressive_speed'}
    exclusive         : set()
  Variables
    /parameters/aggressive_speed : 60.0



In [72]:
import py_trees
class Foo(py_trees.behaviour.Behaviour):

    def __init__(self, name):
        super().__init__(name=name)
        self.blackboard = self.attach_blackboard_client(name="Foo Global")
        self.parameters = self.attach_blackboard_client(name="Foo Params", namespace="foo_parameters_")
        self.state = self.attach_blackboard_client(name="Foo State", namespace="foo_state_")
        self.feedback_message = 0
        

        # create a key 'foo_parameters_init' on the blackboard
        self.parameters.register_key("init", access=py_trees.common.Access.READ)
        # create a key 'foo_state_number_of_noodles' on the blackboard
        self.state.register_key("number_of_noodles", access=py_trees.common.Access.WRITE)

    def initialise(self):
        self.state.number_of_noodles = self.parameters.init

    def update(self):
        self.state.number_of_noodles += 1
        self.feedback_message = self.state.number_of_noodles
        if self.state.number_of_noodles > 5:
            return py_trees.common.Status.SUCCESS
        else:
            return py_trees.common.Status.RUNNING

In [73]:
# could equivalently do directly via the Blackboard static methods if
# not interested in tracking / visualising the application configuration
configuration = py_trees.blackboard.Client(name="App Config")


In [74]:
configuration.register_key("foo_parameters_init", access=py_trees.common.Access.WRITE)

In [75]:
configuration.foo_parameters_init = 3

In [76]:
foo = Foo(name="The Foo")

In [77]:
for i in range(1, 8):
    foo.tick_once()
    print("Number of Noodles: {}".format(foo.feedback_message))

AttributeError: client 'Foo Params' does not have read/write access to '/foo_parameters_/init'